## Extração do site iValor

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def job():
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
    }

    listagem = []
    url = 'https://www.ivalor.com.br/empresas/listagem?p=1'
    
    for page_number in range(1, 29):
        urlbase = url.replace('p=1', 'p=' + str(page_number))

        response = requests.get(urlbase, headers=header)
        site = BeautifulSoup(response.text, 'html.parser')
        tabela = site.find('tbody', id='tab_empresas_body')

        if tabela:
            rows = tabela.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) >= 5:  # Garantir que temos células suficientes
                    Index = cells[0].text.strip()
                    Empresa = cells[1].text.strip()
                    acoes = cells[2].find_all('a')
                    Classificacao = cells[3].text.strip()
                    Setor = cells[4].text.strip()

                    # Expandir os papéis, criando uma nova linha para cada papel
                    for acao in acoes:
                        Papél = acao.text.strip()
                        listagem.append([Index, Empresa, Papél, Classificacao, Setor])

    # Criar o DataFrame
    df = pd.DataFrame(listagem, columns=['Index', 'Empresa', 'Papél', 'Classificacao', 'Setor'])
    
    return df

# Executa a função e obtém o DataFrame
df_auxiliar = job()

# Mostra as primeiras linhas do DataFrame
print(df_auxiliar.head())

# Exporta o DataFrame para um arquivo CSV na pasta atual da aplicação
output_file = "listagem_empresas.csv"
df_auxiliar.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV exportado para: {output_file}")


  Index       Empresa  Papél Classificacao                    Setor
0     1        COMGAS  CGAS3       Mid Cap             Novo Mercado
1     1        COMGAS  CGAS5       Mid Cap             Novo Mercado
2     2          VALE  VALE3      Mega Cap             Novo Mercado
3     3  SANTANDER BR  SANB3       Big Cap  Balcão Org. Tradicional
4     3  SANTANDER BR  SANB4       Big Cap  Balcão Org. Tradicional
Arquivo CSV exportado para: listagem_empresas.csv


## Extração da API yahoo finance

In [25]:
import yfinance as yf
import pandas as pd

# Lista com os 20 principais papéis do mercado brasileiro
principais_papeis = [
    'PETR4.SA',  # Petrobras
    'VALE3.SA',  # Vale
    'ITUB4.SA',  # Itaú Unibanco
    'BBDC4.SA',  # Bradesco
    'BBAS3.SA',  # Banco do Brasil
    'ABEV3.SA',  # Ambev
    'B3SA3.SA',  # B3
    'WEGE3.SA',  # WEG
    'RENT3.SA',  # Localiza
    'JBSS3.SA',  # JBS
    'MGLU3.SA',  # Magazine Luiza
    'BRFS3.SA',  # BRF
    'SUZB3.SA',  # Suzano
    'GGBR4.SA',  # Gerdau
    'CSNA3.SA',  # CSN
    'LREN3.SA',  # Lojas Renner
    'RAIL3.SA',  # Rumo
    'KLBN11.SA', # Klabin
    'EGIE3.SA',  # Engie Brasil
    'ELET3.SA'   # Eletrobras
]

# Lista para armazenar os DataFrames de cada empresa
dados_empresas = []

# Percorrer cada papel na lista de principais papéis
for papel in principais_papeis:
    try:
        # Baixar dados históricos de ações do Yahoo Finance para o ano de 2024
        df = yf.download(papel, start='2024-01-01', end='2024-12-31', progress=False)
        
        # Verificar se o DataFrame está vazio (caso não haja dados)
        if df.empty:
            print(f"Nenhum dado encontrado para {papel}.")
            continue
        
        # Adicionar a coluna 'Papel' ao DataFrame para identificação
        df['Papel'] = papel
        
        # Adicionar o DataFrame da empresa à lista
        dados_empresas.append(df)

    except Exception as e:
        # Captura de erros e exibição de uma mensagem
        print(f"Falha ao baixar dados para {papel}: {e}")

# Concatenar todos os DataFrames em um único DataFrame
if dados_empresas:  # Verifica se há dados para concatenar
    df_final = pd.concat(dados_empresas)
    
    # Remover o sufixo ".SA" da coluna 'Papel'
    df_final['Papel'] = df_final['Papel'].str.replace('.SA', '', regex=False)
    
    # Renomear as colunas para manter o mesmo padrão que o código original
    df_final = df_final.rename(columns={'Open': 'Open', 'High': 'High', 'Low': 'Low', 'Close': 'Close', 'Volume': 'Volume'})

    # Exibir as primeiras linhas do DataFrame final
    print(df_final.head())

    # Opcional: salvar o DataFrame em um arquivo CSV
    df_final.to_csv('dados_empresas_2024.csv', index=False)
else:
    print("Nenhum dado foi coletado.")


                 Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2024-01-02  37.439999  37.889999  37.400002  37.779999  33.281128  24043800   
2024-01-03  37.799999  39.119999  37.669998  38.959999  34.320610  52300200   
2024-01-04  39.119999  39.349998  38.509998  38.630001  34.029907  45344900   
2024-01-05  38.869999  39.139999  38.410000  38.720001  34.109192  35783700   
2024-01-08  38.380001  38.430000  37.610001  38.430000  33.853729  35158100   

            Papel  
Date               
2024-01-02  PETR4  
2024-01-03  PETR4  
2024-01-04  PETR4  
2024-01-05  PETR4  
2024-01-08  PETR4  


## Extração dos dados do site Fundamentus

In [27]:
url = 'https://www.fundamentus.com.br/resultado.php'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)


In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.fundamentus.com.br/resultado.php'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)  # Fazendo a requisição
# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Procurando a tabela de resultados
    table = soup.find('table', {'id': 'resultado'})  # Tabela com o id 'resultado'
    
    # Verificando se a tabela foi encontrada
    if table:
        # Extraindo os cabeçalhos da tabela
        headers = [header.text.strip() for header in table.find_all('th')]
        
        # Extraindo os dados da tabela
        rows = []
        for row in table.find_all('tr')[1:]:  # Pulando o cabeçalho
            rows.append([cell.text.strip() for cell in row.find_all('td')])
        
        # Convertendo para um DataFrame do pandas
        df_fundamentus = pd.DataFrame(rows, columns=headers)
        
        # Exibindo o DataFrame
        print(df_fundamentus)

        # Salvando os dados em CSV, se necessário
        df_fundamentus.to_csv('resultados_fundamentus.csv', index=False, sep=';')
    else:
        print('Tabela não encontrada.')
else:
    print(f'Erro ao acessar o site: {response.status_code}')


     Papel Cotação       P/L  P/VP    PSR Div.Yield P/Ativo P/Cap.Giro P/EBIT  \
0    PORP4    2,40      0,00  0,00  0,000     0,00%   0,000       0,00   0,00   
1    PMET3    0,00      0,00  0,00  0,000     0,00%   0,000       0,00   0,00   
2    MNSA4    0,47      0,00  0,00  0,000     0,00%   0,000       0,00   0,00   
3    MNSA3    0,42      0,00  0,00  0,000     0,00%   0,000       0,00   0,00   
4    CLAN3    0,00      0,00  0,00  0,000     0,00%   0,000       0,00   0,00   
..     ...     ...       ...   ...    ...       ...     ...        ...    ...   
982  UBBR3   18,00  1.466,61  4,77  0,000     0,00%   0,000       0,00   0,00   
983  CEPE3  128,00  1.910,07  5,88  1,143     0,00%   0,691      37,60   8,14   
984  SHOW3    1,02  2.750,45  0,42  0,141     0,00%   0,153       2,52   2,87   
985  SEER3    6,09  2.779,85  0,63  0,416     0,00%   0,223       3,91   2,94   
986  LWSA3    4,17  3.794,23  0,86  1,868     0,00%   0,609       7,03  26,17   

    P/Ativ Circ.Liq  ... EV

## Fazendo o relacionamento entre as bases e exportando o .csv

In [31]:
import pandas as pd

# Supondo que df_final, df_auxiliar e df_fundamentus já estejam definidos anteriormente no seu código.

# Merge com df_auxiliar
df_merged = pd.merge(df_final, df_auxiliar, left_on='Papel', right_on='Papél', how='left')

# Excluir a coluna duplicada 'Papél' (opcional)
df_merged = df_merged.drop(columns=['Papél'])

# Merge com df_fundamentus
df_merged = pd.merge(df_merged, df_fundamentus, left_on='Papel', right_on='Papel', how='left')

# Excluir a coluna duplicada 'Papel' (opcional)
# df_merged = df_merged.drop(columns=['Papel'])

# Exibir o resultado do merge
print(df_merged)

# Exportar o DataFrame para um arquivo CSV
df_merged.to_csv('base_final.csv', index=False)


           Open       High        Low      Close  Adj Close    Volume  Papel  \
0     37.439999  37.889999  37.400002  37.779999  33.281128  24043800  PETR4   
1     37.799999  39.119999  37.669998  38.959999  34.320610  52300200  PETR4   
2     39.119999  39.349998  38.509998  38.630001  34.029907  45344900  PETR4   
3     38.869999  39.139999  38.410000  38.720001  34.109192  35783700  PETR4   
4     38.380001  38.430000  37.610001  38.430000  33.853729  35158100  PETR4   
...         ...        ...        ...        ...        ...       ...    ...   
3755  41.340000  41.369999  39.930000  39.930000  39.930000  14894500  ELET3   
3756  39.700001  39.700001  39.130001  39.549999  39.549999   5776600  ELET3   
3757  39.610001  40.500000  39.430000  39.650002  39.650002   8819600  ELET3   
3758  39.680000  39.849998  39.000000  39.160000  39.160000   6684400  ELET3   
3759  39.509998  39.639999  38.970001  39.500000  39.500000   2030700  ELET3   

     Index     Empresa Classificacao  .